# Analyzing League of Legends Ranked Data

### Importing libraries

In [6]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Retrieving html data

In [7]:
fname = "data.html"

In [8]:
def html_file_to_text(filename):
    # opening and reading file splitting by lines
    file = open(fname,'r')
    data = file.read().splitlines()
    
    # connecting lines in a string
    html_text = ""
    for cell in data:
        html_text += cell
        
    return html_text

In [9]:
html_text = html_file_to_text(fname)

### Separating matches data

In [10]:
def get_match_divs(html_text):
    soup = BeautifulSoup(html_text)
    match_divs = soup.findAll("li", {"class": "game-summary"})
    return match_divs

In [11]:
match_divs = get_match_divs(html_text)

In [12]:
len(match_divs)

915

### Getting each match info

In [13]:
## 817 é o primeiro jogo da Season 10

match_info = match_divs[0].findAll("div", {"class": "binding"})

In [14]:
def get_match_info_data(match_div):
    match_info = match_div.findAll("div", {"class": "binding"})
    match_result = match_div.find("div", {"class", "result-marker"})["class"][0].split('-')[2]
    
    #champion_image_url = match_info[0].find("img")["src"]
    champion_level = match_info[1].text
    champion_summoner_1 = match_info[2].find("div")["data-rg-id"]
    champion_summoner_2 = match_info[3].find("div")["data-rg-id"]
    champion_name = match_info[4].text
    match_map = match_info[5].text
    match_mode = match_info[6].text
    #champion_build = [match_info[7].find("div")["data-rg-id"], match_info[8].find("div")["data-rg-id"],
    #                  match_info[9].find("div")["data-rg-id"], match_info[10].find("div")["data-rg-id"],
    #                  match_info[11].find("div")["data-rg-id"], match_info[12].find("div")["data-rg-id"],
    #                  match_info[13].find("div")["data-rg-id"]]
    champion_kills = match_info[14].text
    champion_deaths = match_info[15].text
    champion_assists = match_info[16].text
    champion_farm = match_info[17].text
    champion_gold = match_info[18].text
    match_duration = match_info[19].text
    match_date = match_info[20].text
    return match_date, match_map, match_mode, match_duration, match_result, champion_name, champion_kills, champion_deaths, champion_assists, champion_farm, champion_gold, champion_level, champion_summoner_1, champion_summoner_2

In [15]:
get_match_info_data(match_divs[0])

('7/9/2020',
 "Summoner's Rift",
 'Normal (Draft Mode)',
 '36:35',
 'defeat',
 'Miss Fortune',
 '7',
 '10',
 '9',
 '164',
 '12.2k',
 '15',
 'SummonerHeal',
 'SummonerFlash')

### Creating DataFrame of Matches

In [16]:
list_of_matches = []
for match_div in match_divs:
    list_of_matches.append(list(get_match_info_data(match_div)))

In [17]:
main_df = pd.DataFrame(list_of_matches, columns = [
    "match_date", "match_map", "match_mode", "match_duration", "match_result", "champ_name", "champ_kills", "champ_deaths", "champ_assists", "champ_farm", "champ_gold", "champ_level", "champ_first_spell", "champ_second_spell"
])

In [18]:
main_df

,match_date,match_map,match_mode,match_duration,match_result,champ_name,champ_kills,champ_deaths,champ_assists,champ_farm,champ_gold,champ_level,champ_first_spell,champ_second_spell
0,7/9/2020,Summoner's Rift,Normal (Draft Mode),36:35,defeat,Miss Fortune,7,10,9,164,12.2k,15,SummonerHeal,SummonerFlash
1,7/9/2020,Howling Abyss,,31:04,victory,Miss Fortune,20,9,33,172,22.1k,18,SummonerHeal,SummonerFlash
2,7/9/2020,Summoner's Rift,Ranked (Draft Mode),36:04,victory,Xayah,12,4,4,269,17.4k,16,SummonerHeal,SummonerFlash
3,7/8/2020,Summoner's Rift,Normal (Draft Mode),30:32,defeat,Miss Fortune,6,6,7,188,11.0k,13,SummonerHeal,SummonerFlash
4,7/8/2020,Summoner's Rift,Normal (Draft Mode),3:20,earlysurrender,Xayah,0,0,1,16,1.2k,3,SummonerHeal,SummonerFlash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,12/8/2019,Howling Abyss,,25:08,victory,Viktor,10,8,36,52,14.9k,18,SummonerMana,SummonerFlash
911,12/7/2019,Summoner's Rift,Normal (Draft Mode),35:42,defeat,Nunu & Willump,3,9,15,156,11.1k,15,SummonerSmite,SummonerFlash
912,12/5/2019,Summoner's Rift,Normal (Draft Mode),36:48,victory,Ezreal,6,10,13,145,12.1k,16,SummonerHeal,SummonerFlash
913,12/5/2019,Summoner's Rift,Normal (Draft Mode),39:12,victory,Lucian,9,6,14,288,18.9k,18,SummonerHeal,SummonerFlash


In [19]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   match_date          915 non-null    object
 1   match_map           915 non-null    object
 2   match_mode          915 non-null    object
 3   match_duration      915 non-null    object
 4   match_result        915 non-null    object
 5   champ_name          915 non-null    object
 6   champ_kills         915 non-null    object
 7   champ_deaths        915 non-null    object
 8   champ_assists       915 non-null    object
 9   champ_farm          915 non-null    object
 10  champ_gold          915 non-null    object
 11  champ_level         915 non-null    object
 12  champ_first_spell   915 non-null    object
 13  champ_second_spell  915 non-null    object
dtypes: object(14)
memory usage: 100.2+ KB


### Fixing champ_gold column

In [20]:
main_df.champ_gold.head()

0    12.2k
1    22.1k
2    17.4k
3    11.0k
4     1.2k
Name: champ_gold, dtype: object

In [21]:
def change_gold_string(gold):
    new_gold = ""
    if gold[-1] == "k":
        aux_gold = gold.split('.')
        new_gold = aux_gold[0] + aux_gold[1][0] + "00"
    else:
        new_gold = gold
    return new_gold

In [22]:
main_df.champ_gold = main_df.champ_gold.apply(change_gold_string)

In [23]:
main_df.head()

,match_date,match_map,match_mode,match_duration,match_result,champ_name,champ_kills,champ_deaths,champ_assists,champ_farm,champ_gold,champ_level,champ_first_spell,champ_second_spell
0,7/9/2020,Summoner's Rift,Normal (Draft Mode),36:35,defeat,Miss Fortune,7,10,9,164,12200,15,SummonerHeal,SummonerFlash
1,7/9/2020,Howling Abyss,,31:04,victory,Miss Fortune,20,9,33,172,22100,18,SummonerHeal,SummonerFlash
2,7/9/2020,Summoner's Rift,Ranked (Draft Mode),36:04,victory,Xayah,12,4,4,269,17400,16,SummonerHeal,SummonerFlash
3,7/8/2020,Summoner's Rift,Normal (Draft Mode),30:32,defeat,Miss Fortune,6,6,7,188,11000,13,SummonerHeal,SummonerFlash
4,7/8/2020,Summoner's Rift,Normal (Draft Mode),3:20,earlysurrender,Xayah,0,0,1,16,1200,3,SummonerHeal,SummonerFlash


### Fixing match_duration column

In [24]:
def change_duration_time(duration):
    new_time = ""
    aux_time = duration.split(':')
    new_time = aux_time[0] + str(int(aux_time[1])/60)[1:]
    return new_time

In [25]:
main_df.match_duration = main_df.match_duration.apply(change_duration_time)

In [26]:
main_df.head()

,match_date,match_map,match_mode,match_duration,match_result,champ_name,champ_kills,champ_deaths,champ_assists,champ_farm,champ_gold,champ_level,champ_first_spell,champ_second_spell
0,7/9/2020,Summoner's Rift,Normal (Draft Mode),36.5833333333333334,defeat,Miss Fortune,7,10,9,164,12200,15,SummonerHeal,SummonerFlash
1,7/9/2020,Howling Abyss,,31.06666666666666667,victory,Miss Fortune,20,9,33,172,22100,18,SummonerHeal,SummonerFlash
2,7/9/2020,Summoner's Rift,Ranked (Draft Mode),36.06666666666666667,victory,Xayah,12,4,4,269,17400,16,SummonerHeal,SummonerFlash
3,7/8/2020,Summoner's Rift,Normal (Draft Mode),30.5333333333333333,defeat,Miss Fortune,6,6,7,188,11000,13,SummonerHeal,SummonerFlash
4,7/8/2020,Summoner's Rift,Normal (Draft Mode),3.3333333333333333,earlysurrender,Xayah,0,0,1,16,1200,3,SummonerHeal,SummonerFlash


### Changing columns dtypes

In [27]:
### date columns
main_df.match_date = pd.to_datetime(main_df.match_date)

### numeric columns
main_df.champ_kills = pd.to_numeric(main_df.champ_kills)
main_df.champ_deaths = pd.to_numeric(main_df.champ_deaths)
main_df.champ_assists = pd.to_numeric(main_df.champ_assists)
main_df.champ_farm = pd.to_numeric(main_df.champ_farm)
main_df.champ_gold = pd.to_numeric(main_df.champ_gold)
main_df.champ_level = pd.to_numeric(main_df.champ_level)
main_df.match_duration = pd.to_numeric(main_df.match_duration)

In [28]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   match_date          915 non-null    datetime64[ns]
 1   match_map           915 non-null    object        
 2   match_mode          915 non-null    object        
 3   match_duration      915 non-null    float64       
 4   match_result        915 non-null    object        
 5   champ_name          915 non-null    object        
 6   champ_kills         915 non-null    int64         
 7   champ_deaths        915 non-null    int64         
 8   champ_assists       915 non-null    int64         
 9   champ_farm          915 non-null    int64         
 10  champ_gold          915 non-null    int64         
 11  champ_level         915 non-null    int64         
 12  champ_first_spell   915 non-null    object        
 13  champ_second_spell  915 non-null    object        

### Separating Season 10 data

In [29]:
season10_df = main_df[:817]

## Final Results

In [30]:
season10_df

,match_date,match_map,match_mode,match_duration,match_result,champ_name,champ_kills,champ_deaths,champ_assists,champ_farm,champ_gold,champ_level,champ_first_spell,champ_second_spell
0,2020-07-09,Summoner's Rift,Normal (Draft Mode),36.583333,defeat,Miss Fortune,7,10,9,164,12200,15,SummonerHeal,SummonerFlash
1,2020-07-09,Howling Abyss,,31.066667,victory,Miss Fortune,20,9,33,172,22100,18,SummonerHeal,SummonerFlash
2,2020-07-09,Summoner's Rift,Ranked (Draft Mode),36.066667,victory,Xayah,12,4,4,269,17400,16,SummonerHeal,SummonerFlash
3,2020-07-08,Summoner's Rift,Normal (Draft Mode),30.533333,defeat,Miss Fortune,6,6,7,188,11000,13,SummonerHeal,SummonerFlash
4,2020-07-08,Summoner's Rift,Normal (Draft Mode),3.333333,earlysurrender,Xayah,0,0,1,16,1200,3,SummonerHeal,SummonerFlash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,2020-01-11,Summoner's Rift,Normal (Draft Mode),21.933333,victory,Rakan,1,3,15,10,7000,12,SummonerDot,SummonerFlash
813,2020-01-10,Summoner's Rift,Ranked (Draft Mode),36.850000,defeat,Lucian,9,6,12,197,14200,15,SummonerHeal,SummonerFlash
814,2020-01-10,Summoner's Rift,Ranked (Draft Mode),28.300000,victory,Aphelios,2,8,19,139,10100,14,SummonerHeal,SummonerFlash
815,2020-01-09,Summoner's Rift,Normal (Draft Mode),34.983333,defeat,Irelia,11,8,7,212,15900,16,SummonerTeleport,SummonerFlash


In [31]:
season10_df.to_csv("season10_data.csv")